# Constructing a knowledge graph using Diffbot

This Jupyter Notebook will guide you step-by-step on how to build a web-powered knowledge graph with Diffbot Natural Language API and LangChain.

Install Required Packages
```bash
pip install --upgrade --quiet  langchain langchain-experimental langchain-openai langchain-neo4j neo4j wikipedia

In [ ]:
# Get your API token from Diffbot. Note that you have to sign up with a work email. 

import os
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
diffbot_api_key = os.getenv("DIFFBOT_API_KEY")
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

The `DiffbotGraphTransformer` class extracts the entities and relationships and returns graph-ready data. Thus, you don’t need dedicated tools for data preprocessing or NER implementation. 


In [ ]:
# Fetch data from Wikipedia

from langchain_community.document_loaders import WikipediaLoader

search_term = "Elon Musk"
wiki_data = WikipediaLoader(query=search_term).load()

try:
    graph_docs = diffbot_nlp.convert_to_graph_documents(wiki_data)
    print("Graph Documents:", graph_docs)
except Exception as e:
    print(f"An error occurred while converting to graph documents: {e}")


In [ ]:
# Start an instance on Neo4j AuraDB and save credentials as environment variables

from langchain_neo4j import Neo4jGraph

uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

graph = Neo4jGraph(uri=uri, username=username, password=password)
graph.add_graph_documents(graph_docs)

# The add_graph_documents method loads the graph documents into Neo4j AuraDB

In [ ]:
#  Query the graph using GraphCypherQAChain and GPT-4.1


from langchain_neo4j import GraphCypherQAChain
from langchain_openai import ChatOpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")
chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model_name="gpt-4.1", api_key=openai_api_key),
    qa_llm=ChatOpenAI(temperature=0, model_name="gpt-4.1", api_key=openai_api_key),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

# Test the chain
result = chain.run("What organizations is Elon Musk a member of?")
print(result)

